In [ ]:
%matplotlib inline


Compute data profiles
=====================


In this example,
we compute the **data profiles** of three algorithms configurations
based on two reference problems.

Imports
-------
We start by making the necessary imports.



In [ ]:
from __future__ import annotations

import shutil
import tempfile
from pathlib import Path

from gemseo import configure
from gemseo.problems.optimization.rastrigin import Rastrigin
from gemseo.problems.optimization.rosenbrock import Rosenbrock

from gemseo_benchmark.algorithms.algorithm_configuration import AlgorithmConfiguration
from gemseo_benchmark.algorithms.algorithms_configurations import (
    AlgorithmsConfigurations,
)
from gemseo_benchmark.data_profiles.target_values import TargetValues
from gemseo_benchmark.problems.problem import Problem
from gemseo_benchmark.problems.problems_group import ProblemsGroup
from gemseo_benchmark.scenario import Scenario

Set the algorithms configurations
---------------------------------
Let us define the algorithms configurations
for which we want to compute data profiles.

For example,
let us choose a configuration of the L-BFGS-B algorithm
with a number of Hessian corrections limited to 2.
(This option is called `maxcor`.)



In [ ]:
lbfgsb_2_corrections = AlgorithmConfiguration(
    "L-BFGS-B",
    "L-BFGS-B with 2 Hessian corrections",
    maxcor=2,
)

Note:
    The customized name `"L-BFGS-B with 2 Hessian corrections"`
    will serve as label in the plot of the data profiles.

To investigate the influence of the ``maxcor`` option,
let us consider a different configuration of L-BFGS-B
with up to 20 Hessian corrections.



In [ ]:
lbfgsb_20_corrections = AlgorithmConfiguration(
    "L-BFGS-B",
    "L-BFGS-B with 20 Hessian corrections",
    maxcor=20,
)

Additionally,
let us choose the SLSQP algorithm,
with all its options set to their default values,
to compare it against L-BFGS-B.



In [ ]:
slsqp_default = AlgorithmConfiguration("SLSQP")

Finally, we gather our selection of algorithms configurations in a group.



In [ ]:
algorithms_configurations = AlgorithmsConfigurations(
    lbfgsb_2_corrections,
    lbfgsb_20_corrections,
    slsqp_default,
    name="Derivative-based algorithms",
)

Set the reference problems
--------------------------
Let us choose two problems already implemented in GEMSEO as references
to measure the performances of our selection of algorithms configurations:
[Rastrigin][gemseo.problems.optimization.rastrigin.Rastrigin]
and [Rosenbrock][gemseo.problems.optimization.rosenbrock.Rosenbrock].

We define target values as an exponential scale of values decreasing towards zero,
the minimum value of both Rastrigin's and Rosenbrock's functions.



In [ ]:
optimum = 0.0
target_values = TargetValues([10**-i for i in range(4, 7)] + [optimum])

Note:
    It could be preferable to customize a different scale of target values
    for each problem, although we keep it simple here.

We now have all the elements to define the benchmarking problems.



In [ ]:
rastrigin = Problem(
    "Rastrigin",
    Rastrigin,
    optimum=optimum,
    doe_size=5,
    doe_algo_name="OT_OPT_LHS",
    target_values=target_values,
)
rosenbrock = Problem(
    "Rosenbrock",
    Rosenbrock,
    optimum=optimum,
    doe_size=5,
    doe_algo_name="OT_OPT_LHS",
    target_values=target_values,
)

Here we configure a design of experiments (DOE)
to generate five starting points by optimized Latin hypercube sampling (LHS).

Finally, we gather our reference problems in a group.



In [ ]:
problems = ProblemsGroup("Reference problems", [rastrigin, rosenbrock])

Generate the benchmarking results
---------------------------------
Now that the algorithms configurations and the reference problems are properly set,
we can measure the performances of the former on the latter.

We set up a [Scenario][gemseo_benchmark.scenario.Scenario] with
our group of algorithms configurations
and a path to a directory where to save the performance histories.



In [ ]:
scenario_dir = Path(tempfile.mkdtemp())
scenario = Scenario([algorithms_configurations], scenario_dir)

Here we choose to deactivate the functions counters, progress bars and bounds check
of GEMSEO to accelerate the script.



In [ ]:
configure(
    enable_function_statistics=False,
    enable_progress_bar=False,
    check_desvars_bounds=False,
)

Let us execute the benchmarking scenario on our group of reference problems.

Note:
    Here we skip the generation of the report
    as we only intend to compute the data profiles.



In [ ]:
results = scenario.execute([problems], skip_report=True)

Compute the datas profiles
--------------------------
Now that the performances histories are generated for the reference problems,
the data profiles of the algorithms configurations can be computed.



In [ ]:
problems.compute_data_profile(algorithms_configurations, results, show=True)

Here we remove the performances histories as we do not wish to keep them.



In [ ]:
shutil.rmtree(scenario_dir)